<a href="https://colab.research.google.com/github/urielmun/Information_protection/blob/main/iot_vuln_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json

# 가중치 설정
WEIGHTS = {
    'DEFAULT_PASSWORD': 30,
    'ADMIN_SERVICES_ACTIVE': 20,   # Telnet/관리포트 (23, 80, 8080 등)
    'WEAK_SSH_AUTH': 15,           # 약한 SSH 인증 방식
    'PUBLIC_ADMIN_INTERFACE': 15,  # 공용 IP에 노출된 관리자 페이지
    'UPNP_SSDP_ACTIVE': 10,        # UPnP/SSDP 활성화
    'FIRMWARE_OUTDATED': 20,       # 구버전 펌웨어
    'KNOWN_VULNERABLE_MODEL': 40,  # 취약 모델
    'WEAK_WIFI_ENCRYPTION': 10,    # 약한 무선 암호화 (WEP 등)
    'GUEST_NETWORK_NOT_ISOLATED': 8,  # 게스트망 미분리
    'PER_OPEN_PORT': 3             # 열린 포트당 가중치
}

# 알려진 취약 모델
KNOWN_VULNERABLE_MODELS = {'CamModel-X1000', 'Router-Z2', 'SmartPlug-A1'}


def calculate_vulnerability_score(metadata, vulnerable_models):
    """IoT 메타데이터 기반 취약점 점수 계산"""
    total_score = 0
    found_vulnerabilities = []

    # 중첩된 키 접근용 헬퍼
    def get_nested(data, keys, default=None):
        for key in keys:
            if not isinstance(data, dict) or key not in data:
                return default
            data = data[key]
        return data

    # 기본 비밀번호
    if get_nested(metadata, ['system', 'password_is_default'], False):
        score = WEIGHTS['DEFAULT_PASSWORD']
        total_score += score
        found_vulnerabilities.append(f"[발견] 기본 비밀번호 사용 (가중치: {score})")

    # Telnet/관리포트 활성화
    if get_nested(metadata, ['services', 'telnet_active'], False) or \
       get_nested(metadata, ['services', 'http_admin_active'], False):
        score = WEIGHTS['ADMIN_SERVICES_ACTIVE']
        total_score += score
        found_vulnerabilities.append(f"[발견] Telnet/관리 서비스 활성화 (가중치: {score})")

    # SSH 인증 방식
    ssh_method = get_nested(metadata, ['services', 'ssh_auth_method'], 'key')
    if ssh_method in ['password', 'default']:
        score = WEIGHTS['WEAK_SSH_AUTH']
        total_score += score
        found_vulnerabilities.append(f"[발견] SSH 취약 인증 방식 ({ssh_method}) (가중치: {score})")

    # 원격 관리자 인터페이스
    if get_nested(metadata, ['network', 'admin_interface_public'], False):
        score = WEIGHTS['PUBLIC_ADMIN_INTERFACE']
        total_score += score
        found_vulnerabilities.append(f"[발견] 공용 IP 관리자 인터페이스 (가중치: {score})")

    # UPnP/SSDP 활성화
    if get_nested(metadata, ['services', 'upnp_active'], False) or \
       get_nested(metadata, ['services', 'ssdp_active'], False):
        score = WEIGHTS['UPNP_SSDP_ACTIVE']
        total_score += score
        found_vulnerabilities.append(f"[발견] UPnP/SSDP 활성화 (가중치: {score})")

    # 펌웨어 버전 비교
    current_fw = get_nested(metadata, ['system', 'firmware_version'])
    latest_fw = get_nested(metadata, ['system', 'latest_firmware_version'])
    if current_fw and latest_fw and current_fw != latest_fw:
        score = WEIGHTS['FIRMWARE_OUTDATED']
        total_score += score
        found_vulnerabilities.append(f"[발견] 펌웨어 구버전 (현재: {current_fw}, 최신: {latest_fw}) (가중치: {score})")

    # 취약 모델
    model = get_nested(metadata, ['system', 'model_name'])
    if model and model in vulnerable_models:
        score = WEIGHTS['KNOWN_VULNERABLE_MODEL']
        total_score += score
        found_vulnerabilities.append(f"[발견] 취약 모델 ({model}) (가중치: {score})")

    # 무선 암호화
    encryption = get_nested(metadata, ['network', 'wireless_encryption'], 'WPA2')
    if encryption.upper() in ['WEP', 'NONE', 'OPEN']:
        score = WEIGHTS['WEAK_WIFI_ENCRYPTION']
        total_score += score
        found_vulnerabilities.append(f"[발견] 약한 무선 암호화 ({encryption}) (가중치: {score})")

    # 게스트 네트워크 분리
    if not get_nested(metadata, ['network', 'guest_network_isolated'], True):
        score = WEIGHTS['GUEST_NETWORK_NOT_ISOLATED']
        total_score += score
        found_vulnerabilities.append(f"[발견] 게스트 네트워크 미분리 (가중치: {score})")

    # 열린 포트 수
    open_ports = get_nested(metadata, ['network', 'open_ports'], [])
    if open_ports:
        port_count = len(open_ports)
        score = port_count * WEIGHTS['PER_OPEN_PORT']
        total_score += score
        found_vulnerabilities.append(f"[발견] 열린 포트 {port_count}개 {open_ports} (가중치: {score})")

    return total_score, found_vulnerabilities


# 예시 데이터
sample_device_metadata = {
    'device_id': 'IoT-CAM-001',
    'network': {
        'admin_interface_public': True,
        'open_ports': [22, 80, 443, 8080],
        'guest_network_isolated': False,
        'wireless_encryption': 'WEP'
    },
    'services': {
        'telnet_active': True,
        'http_admin_active': True,
        'ssh_auth_method': 'password',
        'upnp_active': True,
        'ssdp_active': False
    },
    'system': {
        'password_is_default': True,
        'firmware_version': '1.0.2',
        'latest_firmware_version': '1.5.0',
        'model_name': 'CamModel-X1000'
    }
}

safe_device_metadata = {
    'device_id': 'IoT-LIGHT-002',
    'network': {
        'admin_interface_public': False,
        'open_ports': [443],
        'guest_network_isolated': True,
        'wireless_encryption': 'WPA3'
    },
    'services': {
        'telnet_active': False,
        'http_admin_active': False,
        'ssh_auth_method': 'key',
        'upnp_active': False,
        'ssdp_active': False
    },
    'system': {
        'password_is_default': False,
        'firmware_version': '2.1.0',
        'latest_firmware_version': '2.1.0',
        'model_name': 'SafeLight-2000'
    }
}

# 실행
print("--- 취약한 디바이스 점검 결과 ---")
score1, findings1 = calculate_vulnerability_score(sample_device_metadata, KNOWN_VULNERABLE_MODELS)
print(f"총 가중치: {score1}")
for item in findings1:
    print(f" - {item}")

print("\n" + "---" * 10 + "\n")

print("--- 안전한 디바이스 점검 결과 ---")
score2, findings2 = calculate_vulnerability_score(safe_device_metadata, KNOWN_VULNERABLE_MODELS)
print(f"총 가중치: {score2}")
if not findings2:
    print(" - [취약점 없음]")
else:
    for item in findings2:
        print(f" - {item}")


--- 취약한 디바이스 점검 결과 ---
총 가중치: 180
 - [발견] 기본 비밀번호 사용 (가중치: 30)
 - [발견] Telnet/관리 서비스 활성화 (가중치: 20)
 - [발견] SSH 취약 인증 방식 (password) (가중치: 15)
 - [발견] 공용 IP 관리자 인터페이스 (가중치: 15)
 - [발견] UPnP/SSDP 활성화 (가중치: 10)
 - [발견] 펌웨어 구버전 (현재: 1.0.2, 최신: 1.5.0) (가중치: 20)
 - [발견] 취약 모델 (CamModel-X1000) (가중치: 40)
 - [발견] 약한 무선 암호화 (WEP) (가중치: 10)
 - [발견] 게스트 네트워크 미분리 (가중치: 8)
 - [발견] 열린 포트 4개 [22, 80, 443, 8080] (가중치: 12)

------------------------------

--- 안전한 디바이스 점검 결과 ---
총 가중치: 3
 - [발견] 열린 포트 1개 [443] (가중치: 3)
